In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [30]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(15)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0334, -0.5591, -0.2566]],

        [[ 0.0317, -0.3944,  0.0209]],

        [[ 0.0296, -0.4972, -0.1299]],

        [[ 0.1421, -0.1776, -0.0583]],

        [[ 0.1365, -0.0671, -0.0044]],

        [[ 0.1253, -0.2170,  0.1652]],

        [[ 0.1184, -0.2709,  0.1502]],

        [[ 0.1375, -0.0719,  0.2066]],

        [[ 0.1809, -0.2295,  0.2383]],

        [[ 0.2260, -0.4737,  0.1204]],

        [[ 0.2037, -0.5295,  0.0994]],

        [[ 0.1898, -0.6250,  0.1070]],

        [[ 0.1745, -0.2197,  0.0890]],

        [[ 0.0977, -0.1085, -0.0684]],

        [[ 0.1548, -0.0131,  0.0663]]], grad_fn=<StackBackward>)
(tensor([[[ 0.1548, -0.0131,  0.0663]]], grad_fn=<StackBackward>), tensor([[[ 0.2862, -0.0470,  0.1297]]], grad_fn=<StackBackward>))


In [34]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("I AM PABITRA PRASAD PANI,I AM STAYING AT BANGALORE".split(), ["DET", "NN", "V", "DET", "NN","V","DET", "NN", "V"]),
    ("PABITRA PRASAD PANI IS AN INDIAN & I LOVE MY INDIA".split(), ["NN", "V", "DET", "NN","NN", "V", "DET", "NN","NN", "V", "DET"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'I': 0, 'AM': 1, 'PABITRA': 2, 'PRASAD': 3, 'PANI,I': 4, 'STAYING': 5, 'AT': 6, 'BANGALORE': 7, 'PANI': 8, 'IS': 9, 'AN': 10, 'INDIAN': 11, '&': 12, 'LOVE': 13, 'MY': 14, 'INDIA': 15}


In [35]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [36]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.8344, -1.2175, -1.3097],
        [-0.8862, -1.1202, -1.3411],
        [-0.8555, -1.1808, -1.3172],
        [-0.8371, -1.2247, -1.2976],
        [-0.8655, -1.1348, -1.3562],
        [-0.9133, -1.0858, -1.3426],
        [-0.9490, -0.9884, -1.4242],
        [-0.9844, -0.9810, -1.3808],
        [-0.8919, -1.0827, -1.3805]])
tensor([[-0.2546, -1.5563, -4.2765],
        [-3.6208, -0.1211, -2.4381],
        [-3.6172, -2.4677, -0.1184],
        [-0.2912, -3.4912, -1.5043],
        [-5.6870, -0.0323, -3.5603],
        [-3.1693, -2.4627, -0.1361],
        [-0.1119, -2.6767, -3.2938],
        [-2.6063, -0.1338, -2.9671],
        [-3.9740, -3.3708, -0.0546]])
